Table as_16_q: Asylum seekers in receipt of Section 95 support, by local authority, as at end of quarter

In [ ]:
def is_interactive():
    import __main__ as main
    return not hasattr(main, '__file__')

if is_interactive():
    %run lib/scrape_govuk.ipynb
    metadata = scrape('https://www.gov.uk/government/statistics/immigration-statistics-october-to-december-2017-data-tables')
    import pyexcel
    from io import BytesIO

    ods_files = [f for f in metadata['files']
                 if f['type'] == 'ODS' and
                 f['title'] == 'Asylum data tables immigration statistics October to December 2017 volume 4']
    assert len(ods_files) == 1, 'Should be exactly one matching ODS file'

    ods_url = ods_files[0]['url']
    ods_title = ods_files[0]['title']

    ods_file = BytesIO(session.get(ods_files[0]['url']).content)

    data = pyexcel.get_sheet(file_content=ods_file, file_type='ods', sheet_name='as_16_q')

In [ ]:
df = pd.DataFrame(data.get_array())
df

In [ ]:
observations = df.iloc[2:, :]
observations.rename(columns= observations.iloc[1], inplace=True)
observations.drop(observations.index[0])

In [ ]:
new_table = pd.melt(observations,
                       ['Quarter','Region','Local Authority'],
                       var_name="Total supported under Section 95",
                       value_name="Value")

In [ ]:
new_table = new_table.drop(new_table.index[0])

In [ ]:
new_table.count()

In [ ]:
new_table.dropna(inplace=True)

In [ ]:
new_table = new_table[new_table['Value'] != '.']

In [ ]:
new_table = new_table[new_table['Value'] != 0 ]

In [ ]:
new_table = new_table[new_table['Value'] != 'z']

In [ ]:
new_table.count()

In [ ]:
new_table['Unit'] = 'People'
new_table['Measure Type'] = 'Count'

In [ ]:
new_table.dtypes

In [ ]:
new_table.head()

In [ ]:
new_table = new_table.drop(new_table.index[0])

In [ ]:
new_table['Total supported under Section 95'].unique()

In [ ]:
new_table['Value'] = pd.to_numeric(new_table['Value'], errors='coerce').fillna(0)

In [ ]:
new_table['Value'] = new_table['Value'].astype(int)

In [ ]:
new_table['Period'] = new_table['Quarter'].astype(str)

In [ ]:
new_table.head()

In [ ]:
new_table['Region'] = new_table['Region'].str.lstrip('*')

In [ ]:
new_table['Local Authority'] = new_table['Local Authority'].str.lstrip('*')

In [ ]:
new_table['Period'] = new_table['Period'].map(lambda cell: cell.replace(' ', '-'))

In [ ]:
new_table['Support received'] = new_table['Total supported under Section 95'].map(lambda cell: cell.replace('In receipt of subsistence \nonly', 'In receipt of subsistence only'))

In [ ]:
new_table = new_table[['Period','Region','Local Authority','Support received','Measure Type','Value','Unit']]

In [ ]:
new_table.head()

In [ ]:
new_table = new_table[new_table['Value'] != 0 ]

In [ ]:
destinationFolder = Path('out')
destinationFolder.mkdir(exist_ok=True, parents=True)

new_table.to_csv(destinationFolder / ('as_16_q.csv'), index = False)

writeMetadata(metadata,
              'Asylum seekers receiving support',
              ods_title, 'Migration')

In [ ]:
new_table.count()

In [ ]:
new_table.dtypes